<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/3-LangChain/3_4b_Medical_Assistant_Agent_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>3.4b-Create a Medical Assistant RAG System chat with LangChain, ChromaDB & Gradio</h2>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)

_______
Models: OpenAI.

Colab Environment: CPU.

Keys:
* RAG
* ChromaDB
* Embeddings
* OpenAI
* Gradio
* LangChain
_______
This is the unofficial repository for the book:
        <a href="https://amzn.to/4eanT1g"> <b>Large Language Models:</b> Apply and Implement Strategies for Large Language Models</a> (Apress).
        The book is based on the content of this repository, but the notebooks are being updated, and I am incorporating new examples and chapters.
        If you are looking for the official repository for the book, with the original notebooks, you should visit the
        <a href="https://github.com/Apress/Large-Language-Models-Projects">Apress repository</a>, where you can find all the notebooks in their original format as they appear in the book.

#Installing libraries & Loading Dataset

In [ ]:
!pip install -q langchain==0.3.0
!pip install -q langchain-openai==0.2.0
!pip install -q langchainhub==0.1.21
!pip install -q datasets==3.0.0
!pip install -q chromadb==0.5.5
!pip install -q gradio==4.44.0
!pip install -q langchain-community==0.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 846.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3

We will download the dataset from the Hugging Face datasets library. It's a dataset with information about diseases.

In [ ]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')


README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [ ]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [ ]:
#uncoment this line if you want to limit the size of the data.
data = data[0:100]

As you can see, the medical information in the dataset is well-organized, and to someone like me, who is not an expert in the field, it appears to be quite valuable. This information could be a useful addition to any general medicine book to support primary care doctors.

Load the langchain libraries to load the document.

In [ ]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

The Document is in the Answer column, and the others columns are Metadata.

In [ ]:
df_loader = DataFrameLoader(data, page_content_column="Answer")


In [ ]:
df_document = df_loader.load()
display(df_document[:2])

[Document(metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}, page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.'),
 Document(metadata={'qtype': 'symptoms', 'Question': 'What are the symptoms of Lymphocytic Choriomeningitis (LCM) ?'}, page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become ill, onset of sympt

We can chunk the documents. The size to which we want to split the document is a design decision. The larger it is, the larger the prompt will be, and the slower the Model's response process.

We also need to consider the maximum prompt size and ensure that the document does not exceed it.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1250,
                                      separator="\n",
                                      chunk_overlap=100)
texts = text_splitter.split_documents(df_document)


These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [ ]:
first_doc = texts[1]
print(first_doc.page_content)

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. 
                
For infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain.


# Initialize the Embedding Model and Vector DB

We load the text-embedding-ada-002 model from OpenAI.

In [ ]:
from getpass import getpass
OPENAI_API_KEY = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [ ]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

The execution of this cell may take 3 to 5 minutes. If you want it to be faster, you can reduce the number of records in the dataset.

In [ ]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

We are going to create three objects.

* The language model, which can be any of those from OpenAI.
* The memory, responsible for keeping the prompt with all the necessary history.
* The retrieval, used to obtain information stored in ChromaDB.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm=OpenAI(openai_api_key=OPENAI_API_KEY,
           temperature=0.0)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

We can try the isolated Retrieval to see if the information it returns is relevant.




In [ ]:
qa.invoke("What is the main symptom of LCM?")

{'query': 'What is the main symptom of LCM?',
 'result': ' The main symptom of LCM is a biphasic febrile illness, which includes symptoms such as fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting.'}

Perfect! The information returned is exactly what we desired.

# Creating the Agent.

In [ ]:
from langchain.agents import Tool

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.invoke,
        description=(
            """use this tool when answering medical knowledge queries to get
            more information about the topic"""
        )
    )
]

In [ ]:
from langchain.agents import create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt,
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [ ]:
# Create an agent executor by passing in the agent and tools
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               handle_parsing_errors=True
                               )

## Using the Conversational Agent

To make queries we simply call the `agent` directly.

First i will try a order not related to the Medical field.

In [ ]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: {"query": "area of square", "topic": "geometry"}{'query': '{"query": "area of square", "topic": "geometry"}', 'result': " I don't know."}Do I need to use a tool? No
Final Answer: The area of a square with sides of length 2 is 4 square units.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [],
 'output': 'The area of a square with sides of length 2 is 4 square units.'}

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [ ]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})


Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Clark Kent{'query': 'Clark Kent', 'result': " I don't know."}Do I need to use a tool? No
Final Answer: I am not familiar with Clark Kent. Can you tell me more about them?

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square with sides of length 2 is 4 square units.', additional_kwargs={}, response_metadata={})],
 'output': 'I am not familiar with Clark Kent. Can you tell me more about them?'}

It has not accessed either, as the model has been able to identify that it is not a question related to the database that LangChain provides.

Now it's time to try with a question related to Medicine. Let's see if the model can understand that it should first look for information in the vector database at its disposal.

In [ ]:
 agent_executor.memory.clear()

In [ ]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnosis?"""})


Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Botulism{'query': 'Botulism', 'result': ' Botulism is a rare but serious paralytic illness caused by a nerve toxin produced by certain bacteria. It can be contracted through contaminated food, wounds, or ingestion of bacterial spores. Symptoms include muscle paralysis, difficulty swallowing, and respiratory failure. Treatment includes antitoxin, supportive care, and removal of contaminated food or wound.'}Do I need to use a tool? No
Final Answer: To confirm the diagnosis of Botulism, you can perform a physical exam and order laboratory tests, such as a stool or blood test, to detect the presence of the bacteria or its toxin. It is important to act quickly, as Botulism can be life-threatening if left untreated. 

> Finished chain.


{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnosis?',
 'chat_history': [],
 'output': 'To confirm the diagnosis of Botulism, you can perform a physical exam and order laboratory tests, such as a stool or blood test, to detect the presence of the bacteria or its toxin. It is important to act quickly, as Botulism can be life-threatening if left untreated.'}

Perfect, the most important thing for us is that it has been able to identify that it should go to the medical database to search for information about the symptoms.

In [ ]:
response = agent_executor.invoke({"input": "Is this an important illness?"})


Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Botulism{'query': 'Botulism', 'result': ' Botulism is a rare but serious paralytic illness caused by a nerve toxin produced by certain bacteria. It can be contracted through contaminated food, wounds, or ingestion of bacterial spores. Symptoms include muscle paralysis, difficulty swallowing, and respiratory failure. Treatment includes antitoxin, supportive care, and removal of the source of the toxin.'}Do I need to use a tool? No
Final Answer: Yes, Botulism is a serious illness that requires prompt diagnosis and treatment. It is important to seek medical attention if you suspect you or someone you know may have Botulism.

> Finished chain.


In [ ]:
response['output']

'Yes, Botulism is a serious illness that requires prompt diagnosis and treatment. It is important to seek medical attention if you suspect you or someone you know may have Botulism.'

#Adding Gradio Interface
We are going to create an Interface with Gradio that can work with a LangChain Agent.





In [ ]:
#import gradio library.
import gradio as gr
#history = ""

In [ ]:
# Define the function for the conversation
def continue_conversation(input, history):
    # Invoke the agent and get the response
    response = agent_executor.invoke({"input": input})
    output = response['output']

    # Append the new input and output to the history
    history.append(f"Doctor: {input}")
    history.append(f"AI Assistant: {output}")

    # Join the history into a single string
    history_text = "\n".join(history)

    # Return the current response and the full history (hidden state)
    return output, history_text, history

In [ ]:
#Function call by the clear button to clear the Input textBox.
def clear_input():
    return ""

In [ ]:
# Create the Gradio interface
with gr.Blocks() as demo:
  with gr.Row():
    #We use two columns to organize the Gradio Elements.
    with gr.Column():
      # Input textbox
      input_textbox = gr.Textbox(lines=5, placeholder="Type your prompt here...")
      # Conversation history state
      history_state = gr.State([])

      # Outputs
      current_response = gr.Textbox(label="Current Response")
      conversation_history = gr.Textbox(label="Conversation History", lines=10)
    with gr.Column():
    # Buttons
      send_button = gr.Button("Send")
      clear_button = gr.Button("Clear Input")

      # Bind the send button to submit the input
      send_button.click(
          fn=continue_conversation,
          inputs=[input_textbox, history_state],
          outputs=[current_response, conversation_history, history_state]
      )

      # Bind the clear button to clear the input
      clear_button.click(fn=clear_input, inputs=[], outputs=[input_textbox])

      # Alternatively, pressing "Enter" in the input box will also submit
      input_textbox.submit(
          fn=continue_conversation,
          inputs=[input_textbox, history_state],
          outputs=[current_response, conversation_history, history_state]
      )

In [ ]:
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8c2d7ad81cb3c005b0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)



Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Symptoms: muscle weakness, difficulty swallowing, blurred vision, dry mouth, difficulty breathing{'query': 'Symptoms: muscle weakness, difficulty swallowing, blurred vision, dry mouth, difficulty breathing', 'result': ' These are all symptoms of botulism, a potentially fatal illness caused by a bacterial toxin. Other symptoms may include double vision, drooping eyelids, slurred speech, lethargy, poor feeding, constipation, and weak muscle tone. If left untreated, botulism can lead to paralysis of the respiratory muscles and other parts of the body. Special tests may be needed to diagnose botulism, as other conditions can have similar symptoms. Treatment may involve the use of a breathing machine and intensive medical care, and long-term therapy may be needed for recovery.'} Do I need to use a tool? No
Final Answer: It is important to seek medical attention immediately if you suspect botulism in a patient. Prompt di

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Botulism{'query': 'Botulism', 'result': ' Botulism is a rare but serious paralytic illness caused by a nerve toxin produced by certain bacteria. It can be contracted through contaminated food, wounds, or ingestion of bacterial spores. Symptoms include muscle paralysis, difficulty swallowing, and respiratory failure. Treatment includes antitoxin, supportive care, and removal of contaminated food or wound.'} Do I need to use a tool? No
Final Answer: To confirm the diagnosis of Botulism, you can perform a physical exam and order laboratory tests, such as a stool or blood test, to detect the presence of the bacteria or its toxin. It is important to act quickly, as Botulism can be life-threatening if left untreated.

> Finished chain.


Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Diagnostic tests for Botulism{'query': 'Diagnostic tests for Botulism', 'result': ' Brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), tensilon test for myasthenia gravis, tests for botulinum toxin and bacteria that cause botulism.'}Do I need to use a tool? No
Final Answer: To confirm the diagnosis of Botulism, you can perform a physical exam and order laboratory tests, such as a stool or blood test, to detect the presence of the bacteria or its toxin. It is important to act quickly, as Botulism can be life-threatening if left untreated.

> Finished chain.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8c2d7ad81cb3c005b0.gradio.live


Everything works perfectly, we have an Agent with Memory. We can maintain a conversation, using the Gradio Interface, taking into account that the model knows the previous questions and answers.

# Conclusions.
The experiment has been a small success. The Vectorial database has been configured and filled with information from the dataset. A LangChain agent has been created, and it has been able to retrieve information from the database only when necessary. Don't forget that our ChatBot has memory.

All of this in just a few lines of code!


---